# Commercial Bank Customer Retention Prediction

## APSTA-GE.2401: Statistical Consulting

## Scripts

Created on: 12/04/2020

Modified on: 12/08/2020

## Data Preprocess

### Description

This script preprocesses the raw data originated from the competition data warehouse. There are three main data packages:

- `x_train`: the train data package containing all features.
- `y_train`: the data package for feature test.
- `x_test`: the train data package for prediction. It contains the same features as `x_train`.

### Procedures

We started from examining the `y_train` because it contains labels that can validate our model predictions. `y_train` contains random sampled label data from two quarters: Q3_2019 and Q4_2019. Since the customer ID column, `cust_no`, only contains unique values, we determined our data processing strategies as follows:

1. Use **quarter** to separate data processing procedures. We created two training sets, `X_train_Q3` and `X_train_Q4`, and merged them before applying models. In this way, we bypassed duplicated customer IDs in both `y_train` sets caused by random sampling. This allowed us to maximize the number of labels that can be validated.

    - `y_Q3_3` contains 69126 rows, `y_Q4_3` contains 76170 rows.
    - `y_train` has 62397 duplicated customer IDs. 
    - `y_train` has 40090 completely identical records (same customer ID, same label).
    - Two samples are heavily overlapped.
    - 22307 customers changed their churn preference from Q3 to Q4.
    
2. Based on quarterly-separated `y_train` set, we merged `X_train` raw data accordingly. For each quarter, we dropped duplicated customer IDs except for the last occurrance.

3. During data preprocessing, we examined records in the `cust_avli` column of the `X_train` sets. These sets contain the ID of all effective customers. We confirmed that these ID are the same as those in the `y_train` set. Therefore, we trimmed the dataset based on the `cust_no` column in the `cust_avli`, separated by quarters.

    - Confirmed that `cust_avli` is the key indexing column.
    
4. Merged and Trimmed datasets, saved data to `../data/preprocess`.

### Trimmed Data

After preprocessing, we have three sets:

1. `X`: contains 145296 rows and 55 features.

2. `y`: contains 145296 rows and 1 label column.

3. `X_true`: test set, contains 76722 rows and 55 features.

In [2]:
import pandas as pd
import numpy as np
import csv
import glob
import re
import os

print('SUCCESS! All modules are imported.')

SUCCESS! All modules are imported.


In [157]:
def merge_file(path):
    '''Concatenate files from path
    Param: path: file path from which data are imported and concatenated
    '''
    file_names = [name for name in glob.glob(path)]
    df_temp = (pd.read_csv(file) for file in file_names)
    return(pd.concat(df_temp, ignore_index=True, axis='index'))

----

## y_train

In this step, we proved that there were duplicated customer IDs in each `y_train` sets. We did not perform trimming because even for the same customer, it is possible that he/she changed his/her churn preference after a period of time.

In [261]:
path = '../data/raw/y_train_3/y_Q[34]_3.csv'
y_train = merge_file(path)

In [476]:
y_train.to_csv('../data/preprocess/y_train.csv', index=False)

In [280]:
path = '../data/raw/y_train_3/'
y_Q3_3 = pd.read_csv(path + 'y_Q3_3.csv')
y_Q4_3 = pd.read_csv(path + 'y_Q4_3.csv')

In [281]:
# Index for trimming
idxQ3 = y_Q3_3['cust_no']
idxQ4 = y_Q4_3['cust_no']

In [294]:
print(len(idxQ3))
print(len(idxQ4))

69126
76170


In [287]:
def trim_by_quarter(dat, isQ3):
    '''Trim the data by quarterly index
    Param: dat: imported data
    Param: isQ3: binary, 1 if Q3; 0 if else
    '''
    if isQ3:
        return(dat[dat['cust_no'].isin(idxQ3)])
    else:
        return(dat[dat['cust_no'].isin(idxQ4)])

In [265]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
# Confirm that all customer IDs are unique
print(len(y_Q3_3['cust_no'].value_counts()) == y_Q3_3.shape[0])
print(len(y_Q4_3['cust_no'].value_counts()) == y_Q4_3.shape[0])

True
True


In [266]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('y_Q3_3 has {} rows and {} columns.'.format(y_Q3_3.shape[0], y_Q3_3.shape[1]))
print('y_Q4_3 has {} rows and {} columns.'.format(y_Q4_3.shape[0], y_Q4_3.shape[1]))
print('y_train has {} rows and {} columns.'.format(y_train.shape[0], y_train.shape[1]))
print('y_train has {} duplicated customer IDs.'.format(y_train['cust_no'].duplicated().sum()))
print('y_train has {} completely identical records.'.format(y_train.duplicated().sum()))
print('{} customers changed their churn preference from Q3 to Q4.'.format(
    y_train['cust_no'].duplicated().sum()-y_train.duplicated().sum()))

y_Q3_3 has 69126 rows and 2 columns.
y_Q4_3 has 76170 rows and 2 columns.
y_train has 145296 rows and 2 columns.
y_train has 62397 duplicated customer IDs.
y_train has 40090 completely identical records.
22307 customers changed their churn preference from Q3 to Q4.


----

### Sample Submission File

In [244]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
sample = pd.read_csv('../instructions/sample_submission.csv')
print('Sample submission file has {} rows and {} columns.'.format(sample.shape[0], sample.shape[1]))
display(sample.head())

Sample submission file has 76722 rows and 2 columns.


,cust_no,label
0,0x3b9b4615,0
1,0x3b9ae61b,0
2,0x3b9add69,0
3,0x3b9b3601,0
4,0x3b9b2599,0


----

## X_train

In [461]:
X_train_Q3 = y_Q3_3.drop('label', axis=1).copy()
X_train_Q4 = y_Q4_3.drop('label', axis=1).copy()

### Customer Assets (aum)

In [377]:
# Q3
path = '../data/raw/x_train/aum_m[789].csv'
aum_Q3 = merge_file(path)

In [378]:
# Q4
path = '../data/raw/x_train/aum_m1[012].csv'
aum_Q4 = merge_file(path)

In [379]:
# Drop duplicated customer IDs except for the last occurrance
aum_Q3 = aum_Q3.drop_duplicates(subset=['cust_no'], keep='last')
aum_Q4 = aum_Q4.drop_duplicates(subset=['cust_no'], keep='last')

In [380]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, aum_Q3 has {} rows and {} columns.'.format(aum_Q3.shape[0], aum_Q3.shape[1]))
print('After dropping duplicated customer IDs, aum_Q4 has {} rows and {} columns.'.format(aum_Q4.shape[0], aum_Q4.shape[1]))

After dropping duplicated customer IDs, aum_Q3 has 493441 rows and 9 columns.
After dropping duplicated customer IDs, aum_Q4 has 543823 rows and 9 columns.


In [381]:
# Trim by the customer IDs in `y_train` set, separated by quarter
aum_Q3 = trim_by_quarter(aum_Q3, True)
aum_Q4 = trim_by_quarter(aum_Q4, False)

In [382]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After trimming, aum_Q3 has {} rows and {} columns.'.format(aum_Q3.shape[0], aum_Q3.shape[1]))
print('After trimming, aum_Q4 has {} rows and {} columns.'.format(aum_Q4.shape[0], aum_Q4.shape[1]))

After trimming, aum_Q3 has 69126 rows and 9 columns.
After trimming, aum_Q4 has 76170 rows and 9 columns.


In [386]:
# Save to archive
aum_Q3.to_csv('../data/preprocess/archive/aum_Q3.csv', index=False)
aum_Q4.to_csv('../data/preprocess/archive/aum_Q4.csv', index=False)

In [462]:
# Merge to X_train
X_train_Q3 = X_train_Q3.merge(aum_Q3, how='left', on='cust_no')
X_train_Q4 = X_train_Q4.merge(aum_Q4, how='left', on='cust_no')

### Customer Behavior (behavior)

In [184]:
# Q3
path = '../data/raw/x_train/behavior_m[789].csv'
behavior_Q3 = merge_file(path)

In [185]:
# Q4
path = '../data/raw/x_train/behavior_m1[012].csv'
behavior_Q4 = merge_file(path)

In [186]:
# Drop duplicated customer IDs except for the last occurrance
behavior_Q3 = behavior_Q3.drop_duplicates(subset=['cust_no'], keep='last')
behavior_Q4 = behavior_Q4.drop_duplicates(subset=['cust_no'], keep='last')

In [187]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, behavior_Q3 has {} rows and {} columns.'.format(behavior_Q3.shape[0], behavior_Q3.shape[1]))
print('After dropping duplicated customer IDs, behavior_Q4 has {} rows and {} columns.'.format(behavior_Q4.shape[0], behavior_Q4.shape[1]))

After dropping duplicated customer IDs, behavior_Q3 has 493441 rows and 8 columns.
After dropping duplicated customer IDs, behavior_Q4 has 543823 rows and 8 columns.


In [389]:
# Trim by the customer IDs in `y_train` set, separated by quarter
behavior_Q3 = trim_by_quarter(behavior_Q3, True)
behavior_Q4 = trim_by_quarter(behavior_Q4, False)

In [390]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After trimming, behavior_Q3 has {} rows and {} columns.'.format(behavior_Q3.shape[0], behavior_Q3.shape[1]))
print('After trimming, behavior_Q4 has {} rows and {} columns.'.format(behavior_Q4.shape[0], behavior_Q4.shape[1]))

After trimming, behavior_Q3 has 69126 rows and 8 columns.
After trimming, behavior_Q4 has 76170 rows and 8 columns.


In [391]:
# Save to archive
behavior_Q3.to_csv('../data/preprocess/archive/behavior_Q3.csv', index=False)
behavior_Q4.to_csv('../data/preprocess/archive/behavior_Q4.csv', index=False)

In [463]:
# Merge to X_train
X_train_Q3 = X_train_Q3.merge(behavior_Q3, how='left', on='cust_no')
X_train_Q4 = X_train_Q4.merge(behavior_Q4, how='left', on='cust_no')

### Important Customer Behavior (big_event)

In [189]:
# Q3
path = '../data/raw/x_train/big_event_Q3.csv'
big_event_Q3 = merge_file(path)

<ipython-input-157-cee1144e6e7f>:7: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  return(pd.concat(df_temp, ignore_index=True, axis='index'))


In [190]:
# Q4
path = '../data/raw/x_train/big_event_Q4.csv'
big_event_Q4 = merge_file(path)

In [193]:
# Drop duplicated customer IDs except for the last occurrance
big_event_Q3 = big_event_Q3.drop_duplicates(subset=['cust_no'], keep='last')
big_event_Q3 = big_event_Q4.drop_duplicates(subset=['cust_no'], keep='last')

In [194]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, big_event_Q3 has {} rows and {} columns.'.format(big_event_Q3.shape[0], big_event_Q3.shape[1]))
print('After dropping duplicated customer IDs, big_event_Q4 has {} rows and {} columns.'.format(big_event_Q4.shape[0], big_event_Q4.shape[1]))

After dropping duplicated customer IDs, big_event_Q3 has 493441 rows and 19 columns.
After dropping duplicated customer IDs, big_event_Q4 has 543823 rows and 19 columns.


In [393]:
# Trim by the customer IDs in `y_train` set, separated by quarter
big_event_Q3 = trim_by_quarter(big_event_Q3, True)
big_event_Q4 = trim_by_quarter(big_event_Q4, False)

In [394]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After trimming, big_event_Q3 has {} rows and {} columns.'.format(big_event_Q3.shape[0], big_event_Q3.shape[1]))
print('After trimming, big_event_Q4 has {} rows and {} columns.'.format(big_event_Q4.shape[0], big_event_Q4.shape[1]))

After trimming, big_event_Q3 has 69126 rows and 19 columns.
After trimming, big_event_Q4 has 76170 rows and 19 columns.


In [395]:
# Save to archive
big_event_Q3.to_csv('../data/preprocess/archive/big_event_Q3.csv', index=False)
big_event_Q4.to_csv('../data/preprocess/archive/big_event_Q4.csv', index=False)

In [464]:
# Merge to X_train
X_train_Q3 = X_train_Q3.merge(big_event_Q3, how='left', on='cust_no')
X_train_Q4 = X_train_Q4.merge(big_event_Q4, how='left', on='cust_no')

### Customer Deposits (cunkuan)

In [398]:
# Q3
path = '../data/raw/x_train/cunkuan_m[789].csv'
savings_Q3 = merge_file(path)

In [399]:
# Q4
path = '../data/raw/x_train/cunkuan_m1[012].csv'
savings_Q4 = merge_file(path)

In [400]:
# Drop duplicated customer IDs except for the last occurrance
savings_Q3 = savings_Q3.drop_duplicates(subset=['cust_no'], keep='last')
savings_Q4 = savings_Q4.drop_duplicates(subset=['cust_no'], keep='last')

In [401]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, savings_Q3 has {} rows and {} columns.'.format(savings_Q3.shape[0], savings_Q3.shape[1]))
print('After dropping duplicated customer IDs, savings_Q4 has {} rows and {} columns.'.format(savings_Q4.shape[0], savings_Q4.shape[1]))

After dropping duplicated customer IDs, savings_Q3 has 200721 rows and 3 columns.
After dropping duplicated customer IDs, savings_Q4 has 237049 rows and 3 columns.


In [402]:
# Trim by the customer IDs in `y_train` set, separated by quarter
savings_Q3 = trim_by_quarter(savings_Q3, True)
savings_Q4 = trim_by_quarter(savings_Q4, False)

In [403]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After trimming, savings_Q3 has {} rows and {} columns.'.format(savings_Q3.shape[0], savings_Q3.shape[1]))
print('After trimming, savings_Q4 has {} rows and {} columns.'.format(savings_Q4.shape[0], savings_Q4.shape[1]))

After trimming, savings_Q3 has 69122 rows and 3 columns.
After trimming, savings_Q4 has 76167 rows and 3 columns.


In [404]:
# Save to archive
savings_Q3.to_csv('../data/preprocess/archive/savings_Q3.csv', index=False)
savings_Q4.to_csv('../data/preprocess/archive/savings_Q4.csv', index=False)

In [465]:
# Merge to X_train
X_train_Q3 = X_train_Q3.merge(savings_Q3, how='left', on='cust_no')
X_train_Q4 = X_train_Q4.merge(savings_Q4, how='left', on='cust_no')

### Valid Customer (cust_avli)

Valid customer set contains customer IDs that are same as those in the test set.

In [204]:
# Q3
path = '../data/raw/x_train/cust_avli_Q3.csv'
cust_avli_Q3 = merge_file(path)

In [205]:
# Q4
path = '../data/raw/x_train/cust_avli_Q4.csv'
cust_avli_Q4 = merge_file(path)

In [212]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('cust_avli_Q3 has {} rows and {} columns.'.format(cust_avli_Q3.shape[0], cust_avli_Q3.shape[1]))
print('cust_avli_Q4 has {} rows and {} columns.'.format(cust_avli_Q4.shape[0], cust_avli_Q4.shape[1]))

cust_avli_Q3 has 69126 rows and 1 columns.
cust_avli_Q4 has 76170 rows and 1 columns.


In [211]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
# Confirm that valid customers are the same as those in the test set
print(len(cust_avli_Q3.value_counts()) == y_Q3_3.shape[0])
print(len(cust_avli_Q4.value_counts()) == y_Q4_3.shape[0])

True
True


### Customer Trivias (cust_info)

In [451]:
# Q3
path = '../data/raw/x_train/cust_info_q3.csv'
cust_info_Q3 = merge_file(path)

In [452]:
# Q4
path = '../data/raw/x_train/cust_info_q4.csv'
cust_info_Q4 = merge_file(path)

In [453]:
# Drop duplicated customer IDs except for the last occurrance
cust_info_Q3 = cust_info_Q3.drop_duplicates(subset=['cust_no'], keep='last')
cust_info_Q4 = cust_info_Q4.drop_duplicates(subset=['cust_no'], keep='last')

In [454]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, cust_info_Q3 has {} rows and {} columns.'.format(cust_info_Q3.shape[0], cust_info_Q3.shape[1]))
print('After dropping duplicated customer IDs, cust_info_Q4 has {} rows and {} columns.'.format(cust_info_Q4.shape[0], cust_info_Q4.shape[1]))

After dropping duplicated customer IDs, cust_info_Q3 has 493441 rows and 21 columns.
After dropping duplicated customer IDs, cust_info_Q4 has 543823 rows and 21 columns.


In [455]:
# Trim by the customer IDs in `y_train` set, separated by quarter
cust_info_Q3 = trim_by_quarter(cust_info_Q3, True)
cust_info_Q4 = trim_by_quarter(cust_info_Q4, False)

In [456]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After trimming, cust_info_Q3 has {} rows and {} columns.'.format(cust_info_Q3.shape[0], cust_info_Q3.shape[1]))
print('After trimming, cust_info_Q4 has {} rows and {} columns.'.format(cust_info_Q4.shape[0], cust_info_Q4.shape[1]))

After trimming, cust_info_Q3 has 69126 rows and 21 columns.
After trimming, cust_info_Q4 has 76170 rows and 21 columns.


In [457]:
# Save to archive
cust_info_Q3.to_csv('../data/preprocess/archive/cust_info_Q3.csv', index=False)
cust_info_Q4.to_csv('../data/preprocess/archive/cust_info_Q4.csv', index=False)

In [470]:
# Merge to X_train
X_train_Q3 = X_train_Q3.merge(cust_info_Q3, how='left', on='cust_no')
X_train_Q4 = X_train_Q4.merge(cust_info_Q4, how='left', on='cust_no')

### X_Train Ready

In [471]:
print(X_train_Q3.shape)
print(X_train_Q4.shape)

(69126, 56)
(76170, 56)


In [472]:
X_train_Q3.to_csv('../data/preprocess/archive/X_train_Q3.csv', index=False)
X_train_Q4.to_csv('../data/preprocess/archive/X_train_Q4.csv', index=False)

In [473]:
X_train = X_train_Q3.append(X_train_Q4)

In [474]:
X_train.shape

(145296, 56)

In [475]:
X_train.to_csv('../data/preprocess/X_train.csv', index=False)

----

## X_test

### Customer Assets (aum)

In [220]:
# Q1
path = '../data/raw/x_test/aum_m[123].csv'
aum_Q1 = merge_file(path)

In [221]:
# Drop duplicated customer IDs except for the last occurrance
aum_Q1 = aum_Q1.drop_duplicates(subset=['cust_no'], keep='last')

In [222]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, aum_Q1 has {} rows and {} columns.'.format(aum_Q1.shape[0], aum_Q1.shape[1]))

After dropping duplicated customer IDs, aum_Q1 has 659624 rows and 9 columns.


### Customer Behavior (behavior)

In [246]:
# Q1
path = '../data/raw/x_test/behavior_m[123].csv'
behavior_Q1 = merge_file(path)

In [247]:
# Drop duplicated customer IDs except for the last occurrance
behavior_Q1 = behavior_Q1.drop_duplicates(subset=['cust_no'], keep='last')

In [248]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, behavior_Q1 has {} rows and {} columns.'.format(behavior_Q1.shape[0], behavior_Q1.shape[1]))

After dropping duplicated customer IDs, behavior_Q1 has 659624 rows and 8 columns.


### Important Customer Behavior (big_event)

In [249]:
# Q1
path = '../data/raw/x_test/big_event_Q1.csv'
big_event_Q1 = merge_file(path)

In [250]:
# Drop duplicated customer IDs except for the last occurrance
big_event_Q1 = big_event_Q1.drop_duplicates(subset=['cust_no'], keep='last')

In [251]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, big_event_Q1 has {} rows and {} columns.'.format(big_event_Q1.shape[0], big_event_Q1.shape[1]))

After dropping duplicated customer IDs, big_event_Q1 has 659624 rows and 19 columns.


### Customer Deposits (cunkuan)

In [252]:
# Q1
path = '../data/raw/x_test/cunkuan_m[123].csv'
savings_Q1 = merge_file(path)

In [253]:
# Drop duplicated customer IDs except for the last occurrance
savings_Q1 = savings_Q1.drop_duplicates(subset=['cust_no'], keep='last')

In [254]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, savings_Q1 has {} rows and {} columns.'.format(savings_Q1.shape[0], savings_Q1.shape[1]))

After dropping duplicated customer IDs, savings_Q1 has 254816 rows and 3 columns.


### Valid Customer (cust_avli)

The `cust_avli` in the `x_test` sets is identical to the sample submission file.

In [255]:
# Q1
path = '../data/raw/x_test/cust_avli_Q1.csv'
cust_avli_Q1 = merge_file(path)

In [256]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('cust_avli_Q1 has {} rows and {} columns.'.format(cust_avli_Q1.shape[0], cust_avli_Q1.shape[1]))

cust_avli_Q1 has 76722 rows and 1 columns.


### Customer Trivias (cust_info)

In [257]:
# Q1
path = '../data/raw/x_test/cust_info_q1.csv'
cust_info_Q1 = merge_file(path)

## Drop duplicated customer IDs except for the last occurrance
cust_info_Q1 = cust_info_Q1.drop_duplicates(subset=['cust_no'], keep='last')

In [258]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, cust_info_Q1 has {} rows and {} columns.'.format(cust_info_Q1.shape[0], cust_info_Q1.shape[1]))

After dropping duplicated customer IDs, cust_info_Q1 has 659624 rows and 21 columns.


### X_test Ready

In [418]:
X_test = cust_avli_Q1.copy()

In [420]:
X_test = X_test.merge(aum_Q1, how='left', on='cust_no')

In [421]:
X_test = X_test.merge(behavior_Q1, how='left', on='cust_no')

In [422]:
X_test = X_test.merge(big_event_Q1, how='left', on='cust_no')

In [423]:
X_test = X_test.merge(savings_Q1, how='left', on='cust_no')

In [424]:
X_test = X_test.merge(cust_info_Q1, how='left', on='cust_no')

In [425]:
X_test.shape

(76722, 56)

In [477]:
X_test.to_csv('../data/preprocess/X_test.csv', index=False)

----